In [1]:
%cd ..

d:\~Perso\Etudes\DAC\NeuralNetworksDIY


In [2]:
import numpy as np
from src.convolution import *
from src.linear import *
from src.loss import *
from src.activation import *
from src.encapsulation import *
from utils.mltools import *


def load_usps(fn):
    with open(fn, "r") as f:
        f.readline()
        data = [[float(X) for X in l.split()] for l in f if len(l.split()) > 2]
    tmp = np.array(data)
    return tmp[:, 1:], tmp[:, 0].astype(int)


def get_usps(l, X, y):
    if type(l) != list:
        resx = X[y == l, :]
        resy = y[y == l]
        return resx, resy
    tmp = list(zip(*[get_usps(i, X, y) for i in l]))
    tmpx, tmpy = np.vstack(tmp[0]), np.hstack(tmp[1])
    return tmpx, tmpy


alltrainx, alltrainy = load_usps("data/USPS_train.txt")
alltestx, alltesty = load_usps("data/USPS_test.txt")


def load_one_class(number):
    X_train, y_train = get_usps(number, alltrainx, alltrainy)
    X_test, y_test = get_usps(number, alltestx, alltesty)
    y_train = np.where(y_train == number, -1, 1).reshape(-1, 1)
    y_test = np.where(y_test == number, -1, 1).reshape(-1, 1)
    return X_train, y_train, X_test, y_test


def load_two_classes(neg, pos):
    X_train, y_train = get_usps([neg, pos], alltrainx, alltrainy)
    X_test, y_test = get_usps([neg, pos], alltestx, alltesty)
    y_train = np.where(y_train == neg, -1, 1).reshape(-1, 1)
    y_test = np.where(y_test == neg, -1, 1).reshape(-1, 1)
    return X_train, y_train, X_test, y_test


In [13]:
import os
import pandas as pd
from skimage import io, transform
from PIL import Image


def load_data(root_dir):
    size = (540, 420)
    X = []
    for filepath in os.listdir(root_dir + "/X"):
        imgpath = os.path.join(root_dir, "X", filepath)
        image = io.imread(imgpath)
        image = transform.resize(image, size)
        X.append(image)
    X = np.array(X)

    y = []
    for filepath in os.listdir(root_dir + "/y"):
        imgpath = os.path.join(root_dir, "y", filepath)
        image = io.imread(imgpath)
        image = transform.resize(image, size)
        y.append(image)
    y = np.array(y)

    return X, y


X, y = load_data("./data/dirty_documents")


In [4]:
X[:, np.newaxis, :, :].shape


(144, 1, 540, 420)

In [5]:
X[:, :, :, np.newaxis].shape


(144, 540, 420, 1)

In [6]:
alltrainx.shape

(7291, 256)

In [5]:
alltrainy

array([6, 5, 4, ..., 3, 0, 1])

In [7]:
net = Sequential(
    Conv1D(3, 1, 32, 1),
    MaxPool1D(2, 2),
    Flatten(),
    Linear(4064, 100),
    ReLU(),
    Linear(100, 1),
)
optimizer = Optim(net, CrossEntropyLoss(), eps=1e-3)
lossList = optimizer.SGD(alltrainx[:,:,np.newaxis], alltrainy[:,np.newaxis], 7000, 10)
print(lossList)
pd.Series(lossList).plot()


  0%|          | 0/10 [00:00<?, ?it/s]

Forward de Conv1D
Input : (7291, 256, 1)
Forward de MaxPool1D
Input : (7291, 254, 32)
Forward de Flatten
Input : (7291, 127, 32)
Forward de Linear
Input : (7291, 4064)
Forward de ReLU
Input : (7291, 100)
Forward de Linear
Input : (7291, 100)
Output: (7291, 1)
Shape of loss delta : (7291, 1)
➡️ Backward de Linear
Shape of delta : (7291, 1)
Shape of inputs : (7291, 100)
Backward de Linear ✅
➡️ Backward de ReLU
Shape of delta : (7291, 100)
Shape of inputs : (7291, 100)
Backward de ReLU ✅
➡️ Backward de Linear
Shape of delta : (7291, 100)
Shape of inputs : (7291, 4064)
Backward de Linear ✅
➡️ Backward de Flatten
Shape of delta : (7291, 4064)
Shape of inputs : (7291, 127, 32)
Backward de Flatten ✅
➡️ Backward de MaxPool1D
Shape of delta : (7291, 127, 32)
Shape of inputs : (7291, 254, 32)
Backward de MaxPool1D ✅
➡️ Backward de Conv1D
Shape of delta : (7291, 254, 32)
Shape of inputs : (7291, 256, 1)


 10%|█         | 1/10 [00:15<02:15, 15.10s/it]

Backward de Conv1D ✅
➡️ update_parameters de Conv1D
➡️ update_parameters de MaxPool1D
➡️ update_parameters de Flatten
➡️ update_parameters de Linear
➡️ update_parameters de ReLU
➡️ update_parameters de Linear
Epoch [1], Loss = 0.0453
Forward de Conv1D
Input : (7291, 256, 1)
Forward de MaxPool1D
Input : (7291, 254, 32)
Forward de Flatten
Input : (7291, 127, 32)
Forward de Linear
Input : (7291, 4064)
Forward de ReLU
Input : (7291, 100)
Forward de Linear
Input : (7291, 100)
Output: (7291, 1)
Shape of loss delta : (7291, 1)
➡️ Backward de Linear
Shape of delta : (7291, 1)
Shape of inputs : (7291, 100)
Backward de Linear ✅
➡️ Backward de ReLU
Shape of delta : (7291, 100)
Shape of inputs : (7291, 100)
Backward de ReLU ✅
➡️ Backward de Linear
Shape of delta : (7291, 100)
Shape of inputs : (7291, 4064)
Backward de Linear ✅
➡️ Backward de Flatten
Shape of delta : (7291, 4064)
Shape of inputs : (7291, 127, 32)
Backward de Flatten ✅
➡️ Backward de MaxPool1D
Shape of delta : (7291, 127, 32)
Shape 

## Vectorisation


In [6]:
input = np.arange(25).reshape(5, 5)

kernel_size = 4
layer_stride = 1

height, width = input.shape
rows_stride, columns_strides = input.strides

out_height = int((height - kernel_size) / layer_stride + 1)
out_width = int((width - kernel_size) / layer_stride + 1)

new_shape = (out_height, out_width, kernel_size, kernel_size)
new_strides = (
    rows_stride * layer_stride,
    columns_strides * layer_stride,
    rows_stride,
    columns_strides,
)

windowed_input = np.lib.stride_tricks.as_strided(input, new_shape, new_strides)
print(
    windowed_input,
    f"\nShape: {windowed_input.shape}, \tStrides: {windowed_input.strides}",
)


[[[[ 0  1  2  3]
   [ 5  6  7  8]
   [10 11 12 13]
   [15 16 17 18]]

  [[ 1  2  3  4]
   [ 6  7  8  9]
   [11 12 13 14]
   [16 17 18 19]]]


 [[[ 5  6  7  8]
   [10 11 12 13]
   [15 16 17 18]
   [20 21 22 23]]

  [[ 6  7  8  9]
   [11 12 13 14]
   [16 17 18 19]
   [21 22 23 24]]]] 
Shape: (2, 2, 4, 4), 	Strides: (20, 4, 20, 4)


In [7]:
input = np.arange(5)

kernel_size = 2
layer_stride = 3

length = input.shape[0]
rows_stride = input.strides[0]

out_height = int((length - kernel_size) / layer_stride + 1)

new_shape = (
    out_height,
    kernel_size,
)
new_strides = (
    rows_stride * layer_stride,
    rows_stride,
)

windowed_input = np.lib.stride_tricks.as_strided(input, new_shape, new_strides)
print(
    windowed_input,
    f"\nShape: {windowed_input.shape}, \tStrides: {windowed_input.strides}",
)


[[0 1]
 [3 4]] 
Shape: (2, 2), 	Strides: (12, 4)
